# Imports

In [25]:
import os
import numpy as np
import pandas as pd

# Constants

In [26]:
PROCESSED_FILE_DIRECTORY = r"..\data\excel"

REPORT_FILE_DIRECTORY = r"..\data\report"

# Get Files

In [27]:
files = os.listdir(PROCESSED_FILE_DIRECTORY)

files = [f for f in files if os.path.isfile(PROCESSED_FILE_DIRECTORY + '/' + f)]

# Data Processing

## Sheet 0

In [28]:
def get_empty_dataframe(file):
    data_df = pd.DataFrame({'2021-22': [0],
                                '2020-21': [0],
                                '2019-20': [0],
                                '2018-19': [0],
                                '2017-18': [0],
                                '2016-17': [0],
                                'Institution': file.split('.')[0]
                                })
    return data_df

In [29]:
def process_sheet0():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            if "0" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:            
                #print(f"Processing the sheet 0 of the file: {file}")

                excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                data_df = pd.read_excel(excel_file, sheet_name="0")
                
                data_df = data_df.iloc[:, 1:]
                data_df.replace('-', 0, inplace=True)                

                data_df['2021-22'] = data_df['2021-22'].astype(np.int64)
                data_df['2020-21'] = data_df['2020-21'].astype(np.int64)                                                             
                data_df['2019-20'] = data_df['2019-20'].astype(np.int64)
                data_df['2018-19'] = data_df['2018-19'].astype(np.int64)
                data_df['2017-18'] = data_df['2017-18'].astype(np.int64)
                data_df['2016-17'] = data_df['2016-17'].astype(np.int64)

                data_df['Total'] = data_df['2021-22'] + data_df['2020-21'] + data_df['2019-20'] + \
                data_df['2018-19'] + data_df['2017-18'] + data_df['2016-17']    
            else:
                data_df = get_empty_dataframe(file)
            
            data_df.fillna(0, inplace=True)
            nirf_df = pd.concat([nirf_df, data_df])           
    except Exception as ex:
        print(f"Error while processing the sheet 0 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file)
    finally:
        nirf_df = nirf_df.groupby('Institution').sum('Total')[['Total']]
        nirf_df = nirf_df.transpose()
        nirf_df.rename(index={'Total': 'Total sanctioned approved intake for all programs'}, inplace=True)    

    return nirf_df

### Sheet 1

In [30]:
def get_empty_dataframe(file):
        data_df = pd.DataFrame({'Total Students': [0], 'Institution': file.split('.')[0]})
        
        return data_df

In [31]:
def process_sheet1():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            if "1" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                #print(f"Processing the sheet 1 of the file: {file}")
                excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                data_df = pd.read_excel(excel_file, sheet_name="1")

                data_df = data_df[['Total Students', 'Institution']]
            else:
                data_df = get_empty_dataframe(file)
                
            data_df.fillna(0, inplace=True)        
            nirf_df = pd.concat([nirf_df, data_df])                        
    except Exception as ex:
        print(f"Error while processing the sheet 1 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file) 
    finally:                
        nirf_df = nirf_df.groupby('Institution').sum('Total Students')
        nirf_df = nirf_df.transpose()
        nirf_df.rename(index={'Total Students': 'Total number of students enrolled in all programs'}, inplace=True)        
    
    return nirf_df

## Sheet 2

In [32]:
def get_empty_dataframe(file):
    data_df = pd.DataFrame({'Median salary UG': [0], 'Institution': file.split('.')[0]})
    
    return data_df

In [33]:
def process_sheet2():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            try:
                if "2" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                    #print(f"Processing the sheet 2 of the file: {file}")
                    excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                    data_df = pd.read_excel(excel_file, sheet_name="2")
                    
                    columns = data_df.columns.to_list()
                    median_index = 0
                    institute_index = 0
                    
                    for index, element in enumerate(columns):
                        if element.split()[0] == 'Median':
                            median_index = index                            
                        elif element.split()[0] == 'Institution':
                            institute_index = index

                    data_df = pd.DataFrame({
                        'Median salary UG': data_df.iloc[:, median_index].values,
                        'Institution': data_df.iloc[:, institute_index].values
                    })

                    data_df['Median salary UG'] = data_df['Median salary UG'].str.split(r"(").str[0].astype(np.float64)
                else:
                    data_df = get_empty_dataframe(file)
            except Exception as ex:
                data_df = get_empty_dataframe(file)
            finally:  
                data_df.fillna(0, inplace=True)
                nirf_df = pd.concat([nirf_df, data_df])           
    except Exception as ex:
        print(f"Error while processing the sheet 2 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file)
    finally:            
        nirf_df = nirf_df.groupby('Institution').mean('Median salary UG')
        nirf_df = nirf_df.transpose()           
    
    return nirf_df

## Sheet 3

In [34]:
def get_empty_dataframe(file):
    data_df = pd.DataFrame({'Total': [0], 'Institution': file.split('.')[0]})
    
    return data_df

In [35]:
def process_sheet3():
    nirf_df = pd.DataFrame()
    
    try:    
        for file in files:
            try:
                if "3" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                    #print(f"Processing the sheet 4 of the file: {file}")

                    excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                    data_df = pd.read_excel(excel_file, sheet_name="3")

                    data_df.rename(columns={'Unnamed: 0': 'Total'}, inplace=True)
                    data_df = data_df.iloc[1:3, [1, 4]]
                    data_df['Total'] = data_df['Total'].astype(np.int64) 
                else:
                    data_df = get_empty_dataframe(file)
            except Exception as ex:
                print(f"Error while processing the sheet 3 of the file: {file}")
                print(str(ex))
                data_df = get_empty_dataframe(file)
            finally:    
                data_df.fillna(0, inplace=True)
                nirf_df = pd.concat([nirf_df, data_df])                              
    except Exception as ex:
        print(f"Error while processing the sheet 3 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file)
    finally:            
        nirf_df = nirf_df.groupby('Institution').sum('Total')
        nirf_df = nirf_df.transpose()
        nirf_df.rename(index={'Total': 'Total number of PhD students enrolled'}, inplace=True)
    
    return nirf_df

In [36]:
def process_sheet3_1():
    nirf_df = pd.DataFrame()
    
    try:    
        for file in files:
            try:
                if "3" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                    #print(f"Processing the sheet 4 of the file: {file}")

                    excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                    data_df = pd.read_excel(excel_file, sheet_name="3")

                    data_df = data_df.iloc[[5], 1:]
                    data_df.fillna(0, inplace=True)
                    
                    data_df['Unnamed: 0'] = data_df['Unnamed: 0'].astype(np.float64)
                    data_df['Unnamed: 1'] = data_df['Unnamed: 1'].astype(np.float64)
                    data_df['Unnamed: 2'] = data_df['Unnamed: 2'].astype(np.float64)                    

                    data_df['Total'] = (data_df['Unnamed: 0'] + data_df['Unnamed: 1'] + data_df['Unnamed: 2'])/3              
                    data_df = data_df[['Total', 'Institution']]
                else:
                    data_df = get_empty_dataframe(file, "3")
            except Exception as ex:
                print(f"Error while processing the sheet 3 of the file: {file}")
                print(str(ex))
                data_df = get_empty_dataframe(file)
            finally:    
                data_df.fillna(0, inplace=True)
                nirf_df = pd.concat([nirf_df, data_df])            
    except Exception as ex:
        print(f"Error while processing the sheet 3 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file) 
    finally:            
        nirf_df = nirf_df.set_index('Institution')
        nirf_df.sort_index(inplace=True)
        #nirf_df = nirf_df.groupby('Institution').mean('Total')
        nirf_df = nirf_df.transpose()
        nirf_df.rename(index={'Total': 'Average Full Time PhD students graduated'}, inplace=True)
    
    return nirf_df

## Sheet 4

In [37]:
def get_empty_dataframe(file):
    data_df = pd.DataFrame({
                            '2021-22': [0],
                            '2020-21': [0],
                            '2019-20': [0],
                            'Institution': file.split('.')[0],
                            'Capex1': [0],
                            'Capex2': [0],
                            'Capex3': [0]
                        })
    return data_df

In [38]:
def process_sheet4():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            try:
                if "4" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                    #print(f"Processing the sheet 5 of the file: {file}")

                    excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                    data_df = pd.read_excel(excel_file, sheet_name="4")

                    data_df = data_df.iloc[2:, 1:]
                    data_df.fillna(0, inplace=True)

                    data_df['2021-22'] = data_df['2021-22'].str.split(r"(").str[0].astype(np.int64)

                    data_df['2020-21'] = data_df['2020-21'].str.split(r"(").str[0].astype(np.int64)

                    data_df['2019-20'] = data_df['2019-20'].str.split(r"(").str[0].astype(np.int64)

                    data_df['Capex1'] = round((np.sum(data_df['2021-22'])/100000),1)

                    data_df['Capex2'] = round((np.sum(data_df['2020-21'])/100000),1)

                    data_df['Capex3'] = round((np.sum(data_df['2019-20'])/100000),1)
                else:
                    data_df = get_empty_dataframe(file)
            except Exception as ex:
                print(f"Error while processing the sheet 4 of the file: {file}")
                print(str(ex))
                data_df = get_empty_dataframe(file)
            finally:
                nirf_df = pd.concat([nirf_df, data_df])       
    except Exception as ex:
        print(f"Error while processing the sheet 4 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file)     
    finally:            
        nirf_df = nirf_df[['Capex1', 'Capex2', 'Capex3', 'Institution']].sort_values('Institution')
        nirf_df = nirf_df.set_index('Institution')
        nirf_df = nirf_df.groupby('Institution').head(1)
        nirf_df = nirf_df.transpose()      
    
    return nirf_df

## Sheet 5

In [39]:
def get_empty_dataframe(file):    
    data_df = pd.DataFrame({
                        '2021-22': [0],
                        '2020-21': [0],
                        '2019-20': [0],
                        'Institution': file.split('.')[0],
                        'Opex1': [0],
                        'Opex2': [0],
                        'Opex3': [0]
                    })
    return data_df

In [40]:
def process_sheet5():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            try:
                if "5" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                    #print(f"Processing the sheet 6 of the file: {file}")

                    excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                    data_df = pd.read_excel(excel_file, sheet_name="5")

                    data_df = data_df.iloc[2:, 1:]
                    data_df.fillna(0, inplace=True)
                    
                    data_df['2021-22'] = data_df['2021-22'].str.split(r"(").str[0].astype(np.int64)

                    data_df['2020-21'] = data_df['2020-21'].str.split(r"(").str[0].astype(np.int64)

                    data_df['2019-20'] = data_df['2019-20'].str.split(r"(").str[0].astype(np.int64)

                    data_df['Opex1'] = round((np.sum(data_df['2021-22'])/1000000),1)

                    data_df['Opex2'] = round((np.sum(data_df['2020-21'])/1000000),1)

                    data_df['Opex3'] = round((np.sum(data_df['2019-20'])/1000000),1)                
                else:
                    data_df = get_empty_dataframe(file)
            except Exception as ex:
                print(f"Error while processing the sheet 3 of the file: {file}")
                print(str(ex))
                data_df = get_empty_dataframe(file)
            finally:
                nirf_df = pd.concat([nirf_df, data_df])               
    except Exception as ex:
        print(f"Error while processing the sheet 6 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file)
    finally:            
        nirf_df = nirf_df[['Opex1', 'Opex2', 'Opex3', 'Institution']].sort_values('Institution')
        nirf_df = nirf_df.set_index('Institution')
        nirf_df = nirf_df.groupby('Institution').head(1)
        nirf_df = nirf_df.transpose() 
    
    return nirf_df

## Sheet 6

In [41]:
def get_empty_dataframe(file):         
    data_df = pd.DataFrame({
                        'Sponsored Average': [0], 
                        'Institution': file.split('.')[0]
                    })
    return data_df

In [42]:
def process_sheet6():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            try:
                if "6" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                    #print(f"Processing the sheet 7 of the file: {file}")

                    excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                    data_df = pd.read_excel(excel_file, sheet_name="6")

                    data_df = data_df.iloc[[2], 1:]
                    data_df.fillna(0, inplace=True)
                    
                    data_df['2021-22'] = data_df['2021-22'].astype(np.float64)
                    data_df['2020-21'] = data_df['2020-21'].astype(np.float64)
                    data_df['2019-20'] = data_df['2019-20'].astype(np.float64)

                    data_df['Sponsored Average'] = (data_df['2021-22'] + data_df['2020-21'] + data_df['2019-20'])/3
                    data_df = data_df[['Sponsored Average', 'Institution']]
                else:
                    data_df = get_empty_dataframe(file)
            except Exception as ex:
                print(f"Error while processing the sheet 3 of the file: {file}")
                print(str(ex))
                data_df = get_empty_dataframe(file)
            finally:
                nirf_df = pd.concat([nirf_df, data_df])   
    except Exception as ex:
        print(f"Error while processing the sheet 6 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file)
    finally:            
        nirf_df = nirf_df.groupby('Institution').mean('Sponsored Average')
        nirf_df = nirf_df.transpose() 
    
    return nirf_df

## Sheet 7

In [43]:
def get_empty_dataframe(file):         
    data_df = pd.DataFrame({
                        'Consultancy Average': [0], 
                        'Institution': file.split('.')[0]
                    })
    return data_df

In [44]:
def process_sheet7():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            try:
                if "7" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                    #print(f"Processing the sheet 8 of the file: {file}")

                    excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                    data_df = pd.read_excel(excel_file, sheet_name="7")

                    data_df = data_df.iloc[[2], 1:]
                    data_df.fillna(0, inplace=True)
                    
                    data_df['2021-22'] = data_df['2021-22'].astype(np.float64)
                    data_df['2020-21'] = data_df['2020-21'].astype(np.float64)
                    data_df['2019-20'] = data_df['2019-20'].astype(np.float64)

                    data_df['Consultancy Average'] = (data_df['2021-22'] + data_df['2020-21'] + data_df['2019-20'])/3
                    data_df = data_df[['Consultancy Average', 'Institution']]
                else:
                    data_df = get_empty_dataframe(file)
            except Exception as ex:
                print(f"Error while processing the sheet 7 of the file: {file}")
                print(str(ex))
                data_df = get_empty_dataframe(file)
            finally:
                nirf_df = pd.concat([nirf_df, data_df]) 
    except Exception as ex:
        print(f"Error while processing the sheet 7 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file) 
    finally:        
        nirf_df = nirf_df.groupby('Institution').mean('Consultancy Average')
        nirf_df = nirf_df.transpose()
    
    return nirf_df

# Sheet 8

In [45]:
def get_empty_dataframe(file):         
    data_df = pd.DataFrame({
                        'Executive Development Program Average': [0], 
                        'Institution': file.split('.')[0]
                    })
    return data_df

In [46]:
def process_sheet8():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            try:
                if "8" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                    #print(f"Processing the sheet 8 of the file: {file}")

                    excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                    data_df = pd.read_excel(excel_file, sheet_name="8")

                    data_df = data_df.iloc[[2], :]
                    data_df.fillna(0, inplace=True)
                    
                    data_df['2021-22'] = data_df['2021-22'].astype(np.float64)
                    data_df['2020-21'] = data_df['2020-21'].astype(np.float64)
                    data_df['2019-20'] = data_df['2019-20'].astype(np.float64)

                    data_df['Executive Development Program Average'] = (data_df['2021-22'] +
                                                                        data_df['2020-21'] + 
                                                                        data_df['2019-20'])/3
                    
                    data_df = data_df[['Executive Development Program Average', 'Institution']]
                else:
                    data_df = get_empty_dataframe(file)
            except Exception as ex:
                print(f"Error while processing the sheet 8 of the file: {file}")
                print(str(ex))
                data_df = get_empty_dataframe(file)
            finally:
                nirf_df = pd.concat([nirf_df, data_df]) 
    except Exception as ex:
        print(f"Error while processing the sheet 8 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file) 
    finally:        
        nirf_df = nirf_df.groupby('Institution').mean('Executive Development Program Average')
        nirf_df = nirf_df.transpose()
    
    return nirf_df

In [47]:
process_sheet8()

Institution,IR-M-C-19343,IR-M-C-32855,IR-M-C-32887,IR-M-C-34129,IR-M-C-36557,IR-M-C-37013,IR-M-C-56616,IR-M-I-1044,IR-M-I-1064,IR-M-I-1075,...,IR-M-U-0560,IR-M-U-0564,IR-M-U-0573,IR-M-U-0715,IR-M-U-0735,IR-M-U-0747,IR-M-U-0749,IR-M-U-1004,IR-M-U-1006,IR-M-U-1007
Executive Development Program Average,0.0,1720000.0,0.0,0.0,0.0,904644.0,0.0,0.0,0.0,2.154367e+07,...,0.0,9770333.0,8.311854e+06,31177975.0,0.0,0.0,68393.333333,0.0,0.0,0.0


## Sheet 10

In [48]:
def process_sheet10():
    nirf_df = pd.DataFrame()
    
    try:
        for file in files:
            try:
                if "10" in pd.ExcelFile(PROCESSED_FILE_DIRECTORY + '\\' + file).sheet_names:
                    #print(f"Processing the sheet {sheet_name} of the file: {file}")

                    excel_file = PROCESSED_FILE_DIRECTORY + '\\' + file
                    data_df = pd.read_excel(excel_file, sheet_name="10")
                    data_df.fillna(0, inplace=True)
                else:
                    data_df = get_empty_dataframe(file, "10")
            except Exception as ex:
                print(f"Error while processing the sheet 3 of the file: {file}")
                print(str(ex))
                data_df = get_empty_dataframe(file, "3")
            finally:
                nirf_df = pd.concat([nirf_df, data_df])
    except Exception as ex:
        print(f"Error while processing the sheet 10 of the file: {file}")
        print(str(ex))
        nirf_df = get_empty_dataframe(file, "10")
    finally:
        nirf_df = nirf_df[['Answer', 'Institution']].sort_values('Institution')
        nirf_df = nirf_df.set_index('Institution')
        nirf_df.sort_index(inplace=True)
        nirf_df = nirf_df.transpose()
        nirf_df.rename(index={'Answer': 'Number of faculty members'}, inplace=True)
        
    return nirf_df

In [49]:
nirf_df = pd.DataFrame()

data = process_sheet0()
nirf_df = pd.concat([nirf_df, data])

data = process_sheet1()
nirf_df = pd.concat([nirf_df, data])

data = process_sheet2()
nirf_df = pd.concat([nirf_df, data])

#nirf_df = nirf_df.transpose()
#nirf_df['Median Salary'] = (nirf_df['Median salary UG'] + nirf_df['Median salary PG'])/2
#nirf_df = nirf_df.transpose()

data = process_sheet3()
nirf_df = pd.concat([nirf_df, data])

data = process_sheet3_1()
nirf_df = pd.concat([nirf_df, data])

data = process_sheet4()
nirf_df = pd.concat([nirf_df, data])

nirf_df = nirf_df.transpose()
nirf_df['capexperstudavg'] = (nirf_df['Capex1'] + \
                              nirf_df['Capex2'] + nirf_df['Capex3'])/ \
(3 * (nirf_df['Total sanctioned approved intake for all programs'] + nirf_df['Total number of PhD students enrolled']))
nirf_df = nirf_df.transpose()

data = process_sheet5()
nirf_df = pd.concat([nirf_df, data])

nirf_df = nirf_df.transpose()
nirf_df['opexperstudavg'] = (nirf_df['Opex1'] + \
                              nirf_df['Opex2'] + nirf_df['Opex3'])/ \
(3 * (nirf_df['Total sanctioned approved intake for all programs'] + nirf_df['Total number of PhD students enrolled']))
nirf_df = nirf_df.transpose()

data = process_sheet6()
nirf_df = pd.concat([nirf_df, data])

data = process_sheet7()
nirf_df = pd.concat([nirf_df, data])

data = process_sheet8()
nirf_df = pd.concat([nirf_df, data])

nirf_df

Institution,IR-M-C-19343,IR-M-C-32855,IR-M-C-32887,IR-M-C-34129,IR-M-C-36557,IR-M-C-37013,IR-M-C-56616,IR-M-I-1044,IR-M-I-1064,IR-M-I-1075,...,IR-M-U-0560,IR-M-U-0564,IR-M-U-0573,IR-M-U-0715,IR-M-U-0735,IR-M-U-0747,IR-M-U-0749,IR-M-U-1004,IR-M-U-1006,IR-M-U-1007
Total sanctioned approved intake for all programs,3.600000e+02,6.000000e+02,9.600000e+02,1.320000e+03,480.000000,4.200000e+02,960.000000,4.800000e+02,7.160000e+02,1.600000e+02,...,1.900000e+02,4.500000e+02,4.000000e+02,7.200000e+02,2.700000e+02,1.320000e+03,120.000000,4.600000e+02,4.020000e+02,3.360000e+02
Total number of students enrolled in all programs,3.600000e+02,5.430000e+02,9.450000e+02,1.301000e+03,480.000000,4.150000e+02,968.000000,4.760000e+02,6.480000e+02,1.150000e+02,...,1.380000e+02,5.070000e+02,1.950000e+02,7.130000e+02,2.510000e+02,1.320000e+03,120.000000,4.560000e+02,4.020000e+02,3.250000e+02
Median salary UG,1.910000e+06,7.376667e+05,7.033333e+05,9.433333e+05,578143.333333,5.933333e+05,955666.666667,2.683333e+06,1.298000e+06,1.286667e+06,...,1.235000e+06,5.503333e+05,1.809000e+06,1.553333e+06,9.116667e+05,6.643123e+05,462333.333333,1.091667e+06,1.122667e+06,1.315333e+06
Executive Development Program Average,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total number of PhD students enrolled,1.120000e+02,2.300000e+01,8.000000e+00,6.000000e+00,7.000000,1.400000e+01,52.000000,2.100000e+01,2.300000e+01,8.000000e+01,...,1.270000e+02,1.820000e+02,8.400000e+01,3.400000e+01,1.500000e+01,2.250000e+02,27.000000,6.000000e+00,1.700000e+01,9.000000e+00
Average Full Time PhD students graduated,2.566667e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,3.333333e+00,0.000000,3.333333e-01,6.666667e-01,6.000000e+00,...,5.000000e+00,0.000000e+00,1.433333e+01,3.666667e+00,0.000000e+00,8.333333e+00,2.666667,0.000000e+00,0.000000e+00,0.000000e+00
Capex1,1.047100e+03,1.612000e+02,3.001000e+02,2.616000e+02,102.700000,3.200000e+01,636.900000,1.661700e+03,3.292000e+02,2.454000e+02,...,2.439000e+02,2.469000e+02,2.683000e+02,4.112000e+02,9.900000e+01,7.258000e+02,128.900000,5.366000e+02,9.175400e+03,1.736000e+03
Capex2,4.478000e+02,7.820000e+01,2.651000e+02,4.104000e+02,97.500000,3.940000e+01,362.200000,1.271200e+03,8.200000e+00,1.830000e+02,...,1.048000e+02,2.111000e+02,1.823000e+02,3.561000e+02,5.970000e+01,9.713000e+02,10.800000,2.836000e+02,5.376800e+03,4.162000e+02
Capex3,1.137700e+03,8.230000e+01,2.569000e+02,6.401000e+02,243.200000,9.090000e+01,437.400000,7.417000e+02,1.435000e+02,1.782000e+02,...,1.709000e+02,2.441000e+02,2.874000e+02,4.327000e+02,8.210000e+01,6.481000e+02,32.100000,4.461000e+02,1.670000e+02,4.154000e+02
capexperstudavg,1.859181e+00,1.721241e-01,2.830923e-01,3.298391e-01,0.303491,1.246544e-01,0.473155,2.444844e+00,2.169147e-01,8.425000e-01,...,5.463722e-01,3.703059e-01,5.082645e-01,5.305040e-01,2.816374e-01,5.059763e-01,0.389569,9.057940e-01,1.170979e+01,2.480773e+00


In [51]:
nirf_df.to_excel(REPORT_FILE_DIRECTORY + "/PredictionReport.xlsx", freeze_panes=(1, 1),)